In [58]:
from tree2 import Tree
from anytree import Node, RenderTree
import sys
if sys.version[0] == '2':
    reload(sys)
    sys.setdefaultencoding("utf-8")
import site
from copy import deepcopy
import re
import string
import zss

list1_to_compare=[]
list2_to_compare=[]


def index_2d(myList, v):
    result=[]
    for i, x in enumerate(myList):
        if v in x:
            result.append(i)
            result.append(x.index(v))
            return result

def insert_cost(A):
    #print("inserting "+A.my_label)
    cost=3
    return cost
    
def remove_cost(A):
    #print("deleting "+A.my_label)
    cost=3
    return cost

def shallow_match_parent(A,B):
    if(len(list1_to_compare)>0 & len(list2_to_compare)>0):
        index_A= index_2d(list1_to_compare, A)
        index_B= index_2d(list2_to_compare, B)
        parent_index_A_i=index_A[0]
        parent_index_A_j=int(A.parentID)
        parent_index_B_i=index_B[0]
        parent_index_B_j=int(B.parentID)
        parent_A=list1_to_compare[parent_index_A_i][parent_index_A_j]
        parent_B=list2_to_compare[parent_index_B_i][parent_index_B_j]
        if(parent_A.form==parent_B.form):
            return True
        else:
            return False
    else:
           return False
    
def match_parent(A,B):
    if(len(list1_to_compare)>0 & len(list2_to_compare)>0):
        index_A= index_2d(list1_to_compare, A)
        index_B= index_2d(list2_to_compare, B)
        parent_index_A_i=index_A[0]
        parent_index_A_j=int(A.parentID)
        parent_index_B_i=index_B[0]
        parent_index_B_j=int(B.parentID)
        parent_A=list1_to_compare[parent_index_A_i][parent_index_A_j]
        parent_B=list2_to_compare[parent_index_B_i][parent_index_B_j]
        if((parent_A.form==parent_B.form) & shallow_match_parent(parent_A,parent_B)):
            return True
        else:
            return False
    else:
        return False

def update_cost(A,B):
    incost=0
    if(A.upostag!=B.upostag):
        incost+=1
    if(match_parent(A,B)):
        incost+=1
    if(A.form==B.form):
        cost=incost
    else:
        cost=remove_cost(A)+insert_cost(B)
    return cost

def setPosTag(postag):
    if(postag=="^+G"):
        return "Z"
    if(postag=="N+G"):
        return "S"
    if(postag=="^+V"):
        return "M"
    if(postag=="N+V"):
        return "L"
    if(postag=="O+V"):
        return "L"
    if(postag=="V+R"):
        return "V"
    if(postag=="V+V"):
        return "V"
    if(postag=="X+R"):
        return "Y"

def lookup(postag):
    rplc=""
    if(postag=="NN" or postag=="NNS"):
        rplc= "N"
    if(postag=="PRP" or postag=="WP"):
        rplc= "O"
    if(postag=="NNP" or postag=="NNPS"):
        rplc= "^"
    if(postag[0]=="V" or postag=="MD"):
        rplc= "V"
    if(postag[0]=="J"):
        rplc= "A"
    if(postag[0]=="R" or postag=="WRB"):
        rplc= "R"
    if(postag=="UH" or postag=="UH"):
        rplc= "!"
    answer=re.match(r'WP$',postag)
    answer2=re.match(r'PRP$',postag)
    if(answer!=None or answer2!=None or postag=="DT" or postag=="WDT" or postag=="PRP$"):
        rplc= "D"
    if(postag=="IN" or postag=="TO"):
        rplc="P"
    if(postag=="CC" or postag=="CC"):
        rplc="&"
    if(postag=="RP"):
        rplc="T"
    if(postag=="EX" or postag=="PDT"):
        rplc="X"
    if(postag=="CD"):
        rplc="$"
    if(postag=="FW" or postag=="POS" or postag=="SYM" or postag=="LS"):
        rplc="G"
    if(postag in string.punctuation or postag=="''" or postag=="``" or postag=="-LRB-" or postag=="-RRB-"):
        rplc=","
    if(postag=="HT"):
        rplc="#"
    elif rplc=="":
        rplc= postag
    return rplc

def set_parentID(mergee,merger):
    wordIDlist=[mergee[0],merger[0]]
    parentID=-999
    #print(str(mergee[0])+" "+str(merger[0]))
    #print(str(mergee[6])+" "+str(merger[6]))
    if (mergee[6]==merger[6]):
        parentID= merger[6]
    else:
        if (mergee[6] not in wordIDlist):
            parentID= mergee[6]
        elif (merger[6] not in wordIDlist):
            parentID= merger[6]
    #print("The parentID is "+str(parentID))
    return parentID

def obj_create(tree_b_holder,merge_value,list_to_append):
    if(merge_value):
        updated_b_holder=merge(tree_b_holder)
    else:
        updated_b_holder=tree_b_holder[0]
    

files_to_open=["regular_parser.txt","test.predict"]
for c in range(2):
    tree_holders=[]
    file=files_to_open[c]
    print (c)
    print(file)
    with open(file,encoding="utf-8") as f1:
        hold=f1.read()
        if (c==0):
            trees=hold.split("\n++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n")
            for tree in trees:
                tree_holders.append(tree.split("\n\n"))
        else:
            for element in (hold.split("\n\n")):
                tree=[element]
                tree_holders.append(tree)
    print  (len(tree_holders))      
    i=0
    #print (c)
    tweet_tree_holder=[]   
    for trees in tree_holders:
        tree_b_holder=[]    #tweet level container
        for tree in trees:
            b_holder=[]    #tree level container
            i=0
            sentences=tree.split("\n")
            for sentence in sentences:
                if sentence!="":
                    b=sentence.split("\t")
                    #print(b)
                    if(c==0):
                        b[3]= lookup(b[3])
                        b[4]=b[3]
                        #print(lookup(b[3]))
                        b_holder.append(b) 
                        if(("'" in b[1])& (b[3]!="NNP")):
                                new_form=b_holder[i-1][1]+b[1]
                                new_upostag=b_holder[i-1][3]+'+'+b[3]
                                b_holder[i-1][1]=new_form
                                b_holder[i-1][3]=setPosTag(new_upostag)
                                #merge_position.append(i)
                                #b[6]=str(int(b[6])-merge_count)
                                b_holder[i-1][6]=set_parentID(b_holder[i-1],b)
                                del(b_holder[i])
                                i=i-1
                        i=i+1
                    else:
                        #print(b)
                        b_holder.append(b)
            #print("===>"+str(merge_position))
            tree_b_holder.append(b_holder)
        tweet_tree_holder.append(tree_b_holder)

    for tree_b_holder in tweet_tree_holder:
        if((c==0) & (len(tree_b_holder)>1)):
            obj_create(tree_b_holder,True,c)
        else:
            obj_create(tree_b_holder,False,c)
            
    print("+++++++++++++++++++++++++++++++++")


SyntaxError: invalid syntax (<ipython-input-58-89cf393bc72e>, line 209)